<a href="https://colab.research.google.com/github/cjfghk5697/anomaly-detection-competition/blob/main/inference_code(Add%20TTA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#07-13 TTA, Ensemble 추가
결과적으로 3개의 모델 (대략 55~65%)로 정확도 70퍼로 상승

```python
import ttach as tta
import copy
best_models = []
model.load_state_dict(torch.load('/content/drive/MyDrive/input/model/Focal loss,eff3,47.pth'), strict=False)
best_models.append(copy.deepcopy(model))

apply_tta_model=[]

tta_transforms = tta.Compose(
    [
        tta.Rotate90(angles=[0, 90, 180, 270]),
        tta.Multiply([0.9, 1])
    ]
)
for model in best_models:
   apply_tta_model.append(copytta.deepcopy(ClassificationTTAWrapper(model, tta_transforms)))

tta_model_b3.eval()
tta_model_wrn.eval()
f_pred = []

with torch.no_grad  ():
    for batch in (test_loader):
        x = torch.tensor(batch[0]['image'], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            for model in apply_tta_model:
              pred+ = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/input/"
!unzip -q "/content/drive/MyDrive/input/test.zip"

In [ ]:
!pip3 install timm
!pip install ttach


In [ ]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2
import ttach as tta

import os
import random
import timm

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time
from typing import Tuple, Sequence, Callable
from PIL import Image
import cv2
from autoaugment import ImageNetPolicy, CIFAR10Policy, SVHNPolicy, SubPolicy

device = torch.device('cuda')

In [ ]:
test_png = sorted(glob('/content/drive/MyDrive/input/test/*.png'))

In [ ]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (512, 512))
    return img

In [ ]:
test_imgs = [img_load(n) for n in tqdm(test_png)]

In [ ]:
transforms_test = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [ ]:
train_y = pd.read_csv('/content/drive/MyDrive/input/train_df.csv')

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

In [ ]:
class Custom_dataset(Dataset):
    def __init__(self, 
                 img_paths, 
                 labels, 
                 mode='train',
                 transforms= Sequence[Callable]
            ) -> None:
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
        self.transforms = transforms

    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        if self.mode=='test':
            pass
        img = Image.fromarray(img) # NumPy array to PIL image
        if self.transforms is not None:
            img = self.transforms(img)        
        label = self.labels[idx]
 #       return {'image': img, 'label': label} #TTA사용
        return img, label # 단일 버전
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b3', pretrained=True)
        self.dropout=nn.Dropout(p=0.2)
        self.SiLU=nn.SiLU(inplace=False)
        self.classifier = nn.Linear(1000, 88)
        nn.init.xavier_normal_(self.classifier.weight)

    def forward(self, x):
        x = self.model(x)
        x = self.dropout(x)
        x = self.SiLU(x) 
        x = self.classifier(x)

        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Network().to(device)
model.to(device)# gpu에 모델 할당

import copy
best_models = []
model.load_state_dict(torch.load('/content/drive/MyDrive/input/model/Focal loss,eff3,47.pth'), strict=False)
best_models.append(copy.deepcopy(model))

apply_tta_model=[]

In [ ]:
tta_transforms = tta.Compose(
    [
        tta.Rotate90(angles=[0, 90, 180, 270]),
        tta.Multiply([0.9, 1])
    ]
)
for model in best_models:
   apply_tta_model.append(copytta.deepcopy(ClassificationTTAWrapper(model, tta_transforms)))


In [ ]:
tta_model_b3.eval()
tta_model_wrn.eval()
f_pred = []

with torch.no_grad  ():
    for batch in (test_loader):
        x = torch.tensor(batch[0]['image'], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            for model in apply_tta_model:
              pred+ = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [ ]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/input/sample_submission.csv')

submission["label"] = f_result

submission.to_csv("/content/drive/MyDrive/input/baseline1.csv", index = False)

In [ ]:
model.eval()
f_pred = []

with torch.no_grad():
    for batch in (test_data_loader1):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())
label_decoder = {val:key for key, val in label_unique.items()}
f_result = [label_decoder[result] for result in f_pred]


https://dacon.io/competitions/official/235894/codeshare/4946?page=1&dtype=recent